# B-splines FEM solver for Poisson equation

For this session, we'll need to install the [spl](https://github.com/pyccel/spl) *python* package. Please follow the *README* file installation instructions.

## 1-D Poisson solver

Let's first define the assembly procedures for the stiffness matrix and the rhs.

In [17]:
# needed imports
from spl.linalg.stencil import StencilVector, StencilMatrix

# ... assembling the stiffness matrices using stencil forms
def assembly_matrices(V):

    # ... sizes
    [s1] = V.vector_space.starts
    [e1] = V.vector_space.ends
    [p1] = V.vector_space.pads

    k1 = V.quad_order
    spans_1 = V.spans
    basis_1 = V.basis
    weights_1 = V.weights
    # ...

    # ... data structure
    stiffness = StencilMatrix(V.vector_space, V.vector_space)
    # ...

    # ... build matrices
    for ie1 in range(s1, e1+1-p1):
        i_span_1 = spans_1[ie1]
        for il_1 in range(0, p1+1):
            for jl_1 in range(0, p1+1):
                i1 = i_span_1 - p1  - 1 + il_1
                j1 = i_span_1 - p1  - 1 + jl_1

                v_m = 0.0
                v_s = 0.0
                for g1 in range(0, k1):
                    bi_0 = basis_1[il_1, 0, g1, ie1]
                    bi_x = basis_1[il_1, 1, g1, ie1]

                    bj_0 = basis_1[jl_1, 0, g1, ie1]
                    bj_x = basis_1[jl_1, 1, g1, ie1]

                    wvol = weights_1[g1, ie1]

                    v_m += bi_0 * bj_0 * wvol
                    v_s += (bi_x * bj_x) * wvol

                stiffness[i1, j1 - i1]  += v_s
    # ...

    # ...
    return stiffness
    # ...
# ...

In [18]:
# ... example of assembly of the rhs
def assembly_rhs(V):

    # ... sizes
    [s1] = V.vector_space.starts
    [e1] = V.vector_space.ends
    [p1] = V.vector_space.pads

    k1 = V.quad_order
    spans_1 = V.spans
    basis_1 = V.basis
    points_1 = V.points
    weights_1 = V.weights
    # ...

    # ... data structure
    rhs = StencilVector(V.vector_space)
    # ...

    # ... build rhs
    for ie1 in range(s1, e1+1-p1):
        i_span_1 = spans_1[ie1]
        for il_1 in range(0, p1+1):
            i1 = i_span_1 - p1  - 1 + il_1

            v = 0.0
            for g1 in range(0, k1):
                bi_0 = basis_1[il_1, 0, g1, ie1]
                bi_x = basis_1[il_1, 1, g1, ie1]

                x1    = points_1[g1, ie1]
                wvol  = weights_1[g1, ie1]

                v += bi_0 * 2. * wvol


            rhs[i1] += v
    # ...

    # ...
    return rhs
    # ...
# ...

---
**Note**: 
Although we can put the function as an argument of the *assemble_rhs* function, we will avoid calling a function over the quadrature points for performance reasons. Setting directly the arithemtic expression makes life easier for compilers! You will understand this point in the following section.

---

Now, let's create a B-spline Finite Element Space $V$ over a grid of $32$ elements, and using **cubic** *B-splines*

**TODO put grid as arg of the Space constructor, and remove make_open_knots**

In [19]:
# nedded imports
from spl.core.interface import make_open_knots
from spl.fem.splines import SplineSpace

In [20]:
p  = 3    # spline degree
ne = 32   # number of elements
n  = p + ne
    
# to be removed
knots = make_open_knots(p, n)

# create a finite element space
V = SplineSpace(knots, p)

In [21]:
# assembly
stiffness = assembly_matrices(V)
rhs  = assembly_rhs(V)

In [22]:
# apply homogeneous dirichlet boundary conditions
rhs[0] = 0.
rhs[V.nbasis-1] = 0.

In [23]:
# import cg linear solver
from spl.linalg.solvers import cg

In [24]:
# solve the system
x, info = cg( stiffness, rhs, tol=1e-9, maxiter=1000, verbose=False )

# check
print (info)

{'success': True, 'res_norm': 8.145806060470485e-10, 'niter': 872}


## 2-D Poisson solver

In [25]:
# some imports
from numpy import zeros

In [26]:
def assembly_v0(V):

    # ... sizes
    [s1, s2] = V.vector_space.starts
    [e1, e2] = V.vector_space.ends
    [p1, p2] = V.vector_space.pads
    # ...

    # ... seetings
    [k1, k2] = [W.quad_order for W in V.spaces]
    [spans_1, spans_2] = [W.spans for W in V.spaces]
    [basis_1, basis_2] = [W.basis for W in V.spaces]
    [weights_1, weights_2] = [W.weights for W in V.spaces]
    [points_1, points_2] = [W.points for W in V.spaces]
    # ...

    # ... data structure
    M = StencilMatrix(V.vector_space, V.vector_space)
    # ...

    # ... build matrices
    for ie1 in range(s1, e1+1-p1):
        for ie2 in range(s2, e2+1-p2):
            i_span_1 = spans_1[ie1]
            i_span_2 = spans_2[ie2]
            for il_1 in range(0, p1+1):
                for jl_1 in range(0, p1+1):
                    for il_2 in range(0, p2+1):
                        for jl_2 in range(0, p2+1):

                            i1 = i_span_1 - p1  - 1 + il_1
                            j1 = i_span_1 - p1  - 1 + jl_1

                            i2 = i_span_2 - p2  - 1 + il_2
                            j2 = i_span_2 - p2  - 1 + jl_2

                            v_s = 0.0
                            for g1 in range(0, k1):
                                for g2 in range(0, k2):
                                    bi_0 = basis_1[il_1, 0, g1, ie1] * basis_2[il_2, 0, g2, ie2]
                                    bi_x = basis_1[il_1, 1, g1, ie1] * basis_2[il_2, 0, g2, ie2]
                                    bi_y = basis_1[il_1, 0, g1, ie1] * basis_2[il_2, 1, g2, ie2]

                                    bj_0 = basis_1[jl_1, 0, g1, ie1] * basis_2[jl_2, 0, g2, ie2]
                                    bj_x = basis_1[jl_1, 1, g1, ie1] * basis_2[jl_2, 0, g2, ie2]
                                    bj_y = basis_1[jl_1, 0, g1, ie1] * basis_2[jl_2, 1, g2, ie2]

                                    wvol = weights_1[g1, ie1] * weights_2[g2, ie2]

                                    v_s += (bi_x * bj_x + bi_y * bj_y) * wvol

                            M[i1, i2, j1 - i1, j2 - i2]  += v_s
    # ...

In [27]:
def assembly_v1(V, kernel):

    # ... sizes
    [s1, s2] = V.vector_space.starts
    [e1, e2] = V.vector_space.ends
    [p1, p2] = V.vector_space.pads
    # ...

    # ... seetings
    [k1, k2] = [W.quad_order for W in V.spaces]
    [spans_1, spans_2] = [W.spans for W in V.spaces]
    [basis_1, basis_2] = [W.basis for W in V.spaces]
    [weights_1, weights_2] = [W.weights for W in V.spaces]
    [points_1, points_2] = [W.points for W in V.spaces]
    # ...

    # ... data structure
    M = StencilMatrix(V.vector_space, V.vector_space)
    # ...

    # ... build matrices
    for ie1 in range(s1, e1+1-p1):
        for ie2 in range(s2, e2+1-p2):
            i_span_1 = spans_1[ie1]
            i_span_2 = spans_2[ie2]
            for il_1 in range(0, p1+1):
                for jl_1 in range(0, p1+1):
                    for il_2 in range(0, p2+1):
                        for jl_2 in range(0, p2+1):

                            i1 = i_span_1 - p1  - 1 + il_1
                            j1 = i_span_1 - p1  - 1 + jl_1

                            i2 = i_span_2 - p2  - 1 + il_2
                            j2 = i_span_2 - p2  - 1 + jl_2

                            bi1 = basis_1[il_1, :, :, ie1]
                            bi2 = basis_2[il_2, :, :, ie2]

                            bj1 = basis_1[jl_1, :, :, ie1]
                            bj2 = basis_2[jl_2, :, :, ie2]

                            w1 = weights_1[:, ie1]
                            w2 = weights_2[:, ie2]

                            v_s = kernel(k1, k2, bi1, bi2, bj1, bj2, w1, w2)
                            M[i1, i2, j1 - i1, j2 - i2]  += v_s
    # ...

In [28]:
def kernel_v1(k1, k2, bi1, bi2, bj1, bj2, w1, w2):
    v = 0.0
    for g1 in range(0, k1):
        for g2 in range(0, k2):
            bi_0 = bi1[0, g1] * bi2[0, g2]
            bi_x = bi1[1, g1] * bi2[0, g2]
            bi_y = bi1[0, g1] * bi2[1, g2]

            bj_0 = bj1[0, g1] * bj2[0, g2]
            bj_x = bj1[1, g1] * bj2[0, g2]
            bj_y = bj1[0, g1] * bj2[1, g2]

            wvol = w1[g1] * w2[g2]

            v += (bi_x * bj_x + bi_y * bj_y) * wvol
    return v

In [29]:
def assembly_v2(V, kernel):

    # ... sizes
    [s1, s2] = V.vector_space.starts
    [e1, e2] = V.vector_space.ends
    [p1, p2] = V.vector_space.pads
    # ...

    # ... seetings
    [k1, k2] = [W.quad_order for W in V.spaces]
    [spans_1, spans_2] = [W.spans for W in V.spaces]
    [basis_1, basis_2] = [W.basis for W in V.spaces]
    [weights_1, weights_2] = [W.weights for W in V.spaces]
    [points_1, points_2] = [W.points for W in V.spaces]
    # ...

    # ... data structure
    M = StencilMatrix(V.vector_space, V.vector_space)
    # ...

    # ... element matrix
    mat = zeros((p1+1, p2+1, 2*p1+1, 2*p2+1), order='F')
    # ...

    # ... build matrices
    for ie1 in range(s1, e1+1-p1):
        for ie2 in range(s2, e2+1-p2):
            i_span_1 = spans_1[ie1]
            i_span_2 = spans_2[ie2]

            bs1 = basis_1[:, :, :, ie1]
            bs2 = basis_2[:, :, :, ie2]
            w1 = weights_1[:, ie1]
            w2 = weights_2[:, ie2]
            kernel(p1, p2, k1, k2, bs1, bs2, w1, w2, mat)

            for il_1 in range(0, p1+1):
                for jl_1 in range(0, p1+1):
                    for il_2 in range(0, p2+1):
                        for jl_2 in range(0, p2+1):

                            i1 = i_span_1 - p1  - 1 + il_1
                            j1 = i_span_1 - p1  - 1 + jl_1

                            i2 = i_span_2 - p2  - 1 + il_2
                            j2 = i_span_2 - p2  - 1 + jl_2

                            ij1 = p1 + jl_1 - il_1
                            ij2 = p2 + jl_2 - il_2
                            M[i1, i2, j1 - i1, j2 - i2] += mat[il_1, il_2, ij1, ij2]
    # ...

In [30]:
def kernel_v2(p1, p2, k1, k2, bs1, bs2, w1, w2, mat):
    mat[:,:,:,:] = 0.
    for il_1 in range(0, p1+1):
        for jl_1 in range(0, p1+1):
            for il_2 in range(0, p2+1):
                for jl_2 in range(0, p2+1):

                    v = 0.0
                    for g1 in range(0, k1):
                        for g2 in range(0, k2):
                            bi_0 = bs1[il_1, 0, g1] * bs2[il_2, 0, g2]
                            bi_x = bs1[il_1, 1, g1] * bs2[il_2, 0, g2]
                            bi_y = bs1[il_1, 0, g1] * bs2[il_2, 1, g2]

                            bj_0 = bs1[jl_1, 0, g1] * bs2[jl_2, 0, g2]
                            bj_x = bs1[jl_1, 1, g1] * bs2[jl_2, 0, g2]
                            bj_y = bs1[jl_1, 0, g1] * bs2[jl_2, 1, g2]

                            wvol = w1[g1] * w2[g2]

                            v += (bi_x * bj_x + bi_y * bj_y) * wvol
                    mat[il_1, il_2, p1 + jl_1 - il_1, p2 + jl_2 - il_2] = v

In [31]:
def assembly_v3(V, kernel):

    # ... sizes
    [s1, s2] = V.vector_space.starts
    [e1, e2] = V.vector_space.ends
    [p1, p2] = V.vector_space.pads
    # ...

    # ... seetings
    [k1, k2] = [W.quad_order for W in V.spaces]
    [spans_1, spans_2] = [W.spans for W in V.spaces]
    [basis_1, basis_2] = [W.basis for W in V.spaces]
    [weights_1, weights_2] = [W.weights for W in V.spaces]
    [points_1, points_2] = [W.points for W in V.spaces]
    # ...

    # ... data structure
    M = StencilMatrix(V.vector_space, V.vector_space)
    # ...

    # ... element matrix
    mat = zeros((p1+1, p2+1, 2*p1+1, 2*p2+1), order='F')
    # ...

    # ... build matrices
    for ie1 in range(s1, e1+1-p1):
        for ie2 in range(s2, e2+1-p2):
            i_span_1 = spans_1[ie1]
            i_span_2 = spans_2[ie2]

            bs1 = basis_1[:, :, :, ie1]
            bs2 = basis_2[:, :, :, ie2]
            w1 = weights_1[:, ie1]
            w2 = weights_2[:, ie2]
            kernel(p1, p2, k1, k2, bs1, bs2, w1, w2, mat)

            s1 = i_span_1 - p1 - 1
            s2 = i_span_2 - p2 - 1
            M._data[s1:s1+p1+1,s2:s2+p2+1,:,:] += mat[:,:,:,:]
    # ...

In [32]:
# needed imports
from spl.core.interface import make_open_knots
from spl.fem.splines import SplineSpace
from spl.fem.tensor  import TensorSpace
import time

# ... numbers of elements and degres
p1  = 3 ; p2  = 3
ne1 = 64 ; ne2 = 64
n1 = p1 + ne1 ;  n2 = p2 + ne2
# ...

print('> Grid   :: [{ne1},{ne2}]'.format(ne1=ne1, ne2=ne2))
print('> Degree :: [{p1},{p2}]'.format(p1=p1, p2=p2))

knots_1 = make_open_knots(p1, n1)
knots_2 = make_open_knots(p2, n2)

V1 = SplineSpace(knots_1, p1)
V2 = SplineSpace(knots_2, p2)

V = TensorSpace(V1, V2)

print('-- elapsed times --')
# ... pure python version 0
tb = time.time()
assembly_v0(V)
te = time.time()
print('> {} [pure Python v0]'.format(te-tb))
# ...

# ... pure python version 1
tb = time.time()
assembly_v1(V, kernel_v1)
te = time.time()
print('> {} [pure Python v1]'.format(te-tb))
# ...

# ... pure python version 2
tb = time.time()
assembly_v2(V, kernel_v2)
te = time.time()
print('> {} [pure Python v2]'.format(te-tb))
# ...

# ... pure python version 3 (kernel_3 = kernel_2)
tb = time.time()
assembly_v3(V, kernel_v2)
te = time.time()
print('> {} [pure Python v3]'.format(te-tb))
# ...

> Grid   :: [64,64]
> Degree :: [3,3]
-- elapsed times --
> 64.34922337532043 [pure Python v0]
> 57.50900363922119 [pure Python v1]
> 75.51311111450195 [pure Python v2]
> 1.3250715732574463 [pure Python v3]


In [33]:
# import epyccel function
from pyccel.epyccel import epyccel

# compile the kernel
header_v2 = '#$ header procedure kernel_v2(int, int, int, int, double [:,:,:], double [:,:,:], double [:], double [:], double [:,:,:,:])'
kernel = epyccel(kernel_v2, header_v2)

In [34]:
# ... pyccelized version 3 (kernel_3 = kernel_2)
tb = time.time()
assembly_v3(V, kernel)
te = time.time()
print('> {} [Python-Pyccel v3]'.format(te-tb))
# ...

> 0.006407499313354492 [Python-Pyccel v3]


In [3]:
# css style
from IPython.core.display import HTML
def css_styling():
    styles = open("../../styles/custom.css", "r").read()
    return HTML(styles)
css_styling()